<a href="https://colab.research.google.com/github/otanet/Quantum_Computing_Annealing_Machine_20211209/blob/main/n_queens_20211209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# n-クイーン問題

数独のように、制約条件のみを課すことによって解くことができる最適化問題の例として、エイト・クイ―ンというゲームを一般化した n-クイーン問題の解法について解説します。

## n-クイーン問題のルール

[エイト・クイーン](https://ja.wikipedia.org/wiki/%E3%82%A8%E3%82%A4%E3%83%88%E3%83%BB%E3%82%AF%E3%82%A4%E3%83%BC%E3%83%B3)は、チェス盤上で8個のクイーンをどの駒からも取られることのないように配置するゲームです。
チェス盤は $8\times8$ のグリッドで、クイーンは上下左右斜めの八方向いずれかにいくらでも進むことができます。 ([クイーンの動き](https://ja.wikipedia.org/wiki/%E3%82%AF%E3%82%A4%E3%83%BC%E3%83%B3_(%E3%83%81%E3%82%A7%E3%82%B9)))。

このゲームを一般化した「n-クイーン」は、$n\times n$ のグリッド上で $n$ 個のクイーンを他のどの駒からも取られることのないように配置するゲームです。クイーンは $8\times8$ のチェス盤の場合と同じ動きをするものとします。

ここでは、n-クイーンをアニーリングマシンによる最適化によって解く方法を扱います。

## n-クイーンの定式化

まず、アニーリングマシンでこの問題を扱うために、ゲームのルールを QUBO 変数を用いた定式化を行います。

n-クイーンは以下の2つのタイプのルールによって表現されます。

1. クイーンの数は $n$ 個
2. 各クイーンが動ける範囲には他のクイーンが配置されない

次に、このルールをQUBO 変数を用いた制約条件として表すことを考えます。
まず、$n\times n$ のチェス盤の各マスに1つの変数を割り当てます。マスの位置を2つのインデックス $i,j=0,1,\cdots, n-1$ を用いて $(i, j)$ と表し、そのマスに対応する変数を $q_{i, j}$ とします。
QUBO 変数の意味は、$(i, j)$ のマスにクイーンが配置されている場合は$q_{i,j}=1$とし、$(i, j)$ のマスにどの駒も配置されていない場合は $q_{i,j}=0$ とします。

ルール 1 は、盤面上の全ての変数を足し上げると、その合計は $n$ となるという制約条件と表すことができます。これは、$q_{i,j}=1$ となる QUBO 変数の数を数えたことに相当するので、盤面上のクイーンの数と等価です。

ルール 2 は、$(i, j)$ のマスにクイーンが配置されている場合に、そのクイーンが動けるマスに対応する QUBO 変数を足し上げると $0$ となる制約条件として表すと、動けるマスに他のクイーンが一つも配置されていない条件と等価です。$(i, j)$ にクイーンが配置されていない場合は、特に制約条件を課す必要はありませんが、以下のようにして注意して扱う必要があります。
マス $(i, j)$ にクイーンが「配置されている」/「配置されていない」の2つの場合を同時に表すには、$q_{i,j}$ に $(i, j)$ からクイーンが動けるマスに対応する変数の和を掛けたものが $0$ となる制約条件とします。$q_{i,j}=1$ の場合はそれらのマスの中に他のクイーンが一つもない条件となり、$q_{i,j}=0$ の場合はその値はそもそも $0$ となるので恒真的な条件式が満たされます。

したがって、上記のルールの QUBO 変数による表現をまとめると、

$$
\left\{
    \begin{aligned}
        &\begin{split}
            1. \quad &\sum_{i, j=0}^{n-1} q_{i,j}=n
        \end{split}\\
        &\begin{split}
            2. \quad &q_{i,j}\times\left(\sum_{(k,l)\in M(i,j)}q_{k,l}\right) =0 \quad \text{for }(i,j)\in \text{ 全てのマス}
        \end{split}\\
    \end{aligned}
\right.
$$

となります。ここで、マス $(i,j)$ からクイーンが動ける全てのマスに対応するインデックスの集合を $M(i,j)$ としました。2. の条件については、全てのマスに対応する $(i,j)$ について、上記の条件を課すものとします。

これで定式化が完了したので、次に実装を行います。


## 実装

まず、$n=8$ として、チェス盤面のそれぞれのマスに対応する $n\times n$ の array 型のQUBO 変数を以下のようにして生成します。

In [ ]:
! pip install -q amplify
from amplify import BinarySymbolGenerator

n = 8
q = BinarySymbolGenerator().array(n, n)

上記の QUBO 変数を用いて、制約条件式を構築します。
制約条件 2 では、マス $(i, j)$ にあるクイーンが動けるマスに対応する QUBO 変数の和を取る必要があるので、$(i, j)$ を引数として前述の QUBO 変数の和を計算する `queen_moves` という関数を用意しておくと便利です。

制約条件 1 では、`equal_to` を用いて、全てのマスの変数の和が $n$ になるように制約条件を課します。
制約条件 2 では、`equal_to` を用いて、定式化で与えられた式が $0$ になるように制約条件を課します。

実装は以下のようになります。

In [ ]:
from amplify import sum_poly
from amplify.constraint import equal_to, penalty


def queen_moves(i, j, q):
    """(i,j)からクイーンの動けるマスに対応する QUBO 変数の和を返却

    Args:
        i ([int]): チェス盤の横方向を指定する 0, .., n-1 のインデックス
        j ([int]): チェス盤の縦方向を指定する 0, .., n-1 のインデックス
        q ([amplify.BinaryPolyArray]): n x n の amplify.BinaryPolyArray で与えられる QUBO 変数

    Returns:
        [amplify.BinaryPoly]: (i,j)からクイーンの動けるマスに対応する QUBO 変数の和
    """
    N = len(q)
    moves = sum_poly([q[i, k] for k in range(N) if k != j])  # 左右
    moves += sum_poly([q[k, j] for k in range(N) if k != i])  # 上下
    moves += sum_poly(
        [
            q[i + k, j + k]
            for k in range(-N, N)
            if k != 0 and 0 <= i + k and i + k < N and 0 <= j + k and j + k < N
        ]
    )  # 斜め右下がり
    moves += sum_poly(
        [
            q[i + k, j - k]
            for k in range(-N, N)
            if k != 0 and 0 <= i + k and i + k < N and 0 <= j - k and j - k < N
        ]
    )  # 斜め右上がり

    return moves


c_num_queens = equal_to(q.sum(), n)  # 制約条件 1
c_queen_moves = [
    equal_to(q[i, j] * queen_moves(i, j, q), 0) for i in range(n) for j in range(n)
]  # 制約条件 2

constraints = c_num_queens + sum(c_queen_moves)  # 全ての制約条件の和

次にクライアントとソルバの設定です。
n-クイーンでは、コスト関数を導入せずに制約条件を課すことのみで最適化問題を解くので、制約条件を満たす解のエネルギー値は全て $0$ となります。
Amplify SDK で、同じエネルギー値を持つ解を全て出力するためには、[`client.parameters.outputs.duplicate = True`](https://amplify.fixstars.com/ja/docs/reference/generated/amplify.client.FixstarsClientParametersOutputs.html?highlight=duplicate#amplify.client.FixstarsClientParametersOutputs.duplicate) に設定する必要があります。
この問題では、複数のエネルギー値 $0$ の解が存在することが想定されるので、この設定を行い全ての解を見つけることを考えます。

以下ではタイムアウトを1秒に設定しましたが、場合に応じでタイムアウトの値を十分に大きく取る必要があります。この問題の場合は、1秒のタイムアウトで、現状では $n=10$ 程度までのサイズの問題の全ての解を見つけることができます。

In [ ]:
from amplify import Solver
from amplify.client import FixstarsClient

client = FixstarsClient()
client.token = "トークンを入力してください"
client.parameters.timeout = 1000  # タイムアウト1秒
client.parameters.outputs.duplicate = True  # 制約条件を満たすエネルギー 0 の解を全て出力

solver = Solver(client)

ソルバの設定が済めば、先ほど構築した `constraints` を `solver` の `solver` メソッドに渡すことで、解を得ることができます。

In [ ]:
result = solver.solve(constraints)

if len(result.solutions) == 0:
    print("解が1つも見つかりませんでした。")
else:
    print(f"解の個数: {len(result.solutions)}")

[エイト・クイーンのウェブサイト](https://ja.wikipedia.org/wiki/%E3%82%A8%E3%82%A4%E3%83%88%E3%83%BB%E3%82%AF%E3%82%A4%E3%83%BC%E3%83%B3)によると、$n=8$ の場合は $92$ 個の解がありますが、上の結果からも解の個数が $92$ となることが確認できます。

得られた解がどのような盤面に対応するかをチェックするには、

In [ ]:
def board(q_values):
    """QUBO変数とその解から、盤面を可視化する

    Args:
        q_values ([numpy.ndarray]): N x N の QUBO 変数 array に解を代入したもの
    """
    N = len(q_values)
    sol_str = (
        "".join(["".join(map(str, map(int, q_values[i]))) for i in range(N)])
        .replace("1", "♕")
        .replace("0", "　")
    )
    hor = "     ".join([chr(ord("ａ") + i) for i in range(N)])
    print(f"       {hor}")
    print(f"    {''.join(['-' for _ in range(7 * N + 1)])}")
    for i in range(N):
        line = f"{N - i}".rjust(2)
        line += "  |  "
        line += "  |  ".join(list(sol_str[i * N : (i + 1) * N]))
        line += "  |"
        print(line)
        print(f"    {''.join(['-' for _ in range(7 * N + 1)])}")


#
# 全ての解を QUBO 変数の array に代入し、リストに格納
#
q_values_list = [q.decode(sol.values) for sol in result.solutions]

例えば、$1$ 番目の解を確認したい場合は、以下のようにします。

In [ ]:
board(q_values_list[1])

上記の例は、$n=8$ に設定した エイト・クイーンの場合でしたが、$n$ の値を変更し、n-クイーン問題の解が実際に得られるかを確認してみましょう。